# Poisson Causal Discovery Example

This notebook demonstrates causal network discovery using the **Poisson** information method with synthetic count data.

## Overview
- Generate synthetic Poisson time series with known causal structure
- Visualize the count dynamics and network structure
- Apply causal discovery using Poisson conditional mutual information
- Evaluate performance using ROC-AUC metric

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Import causal discovery components
from causalentropy.core.discovery import discover_network
from causalentropy.datasets.synthetic import poisson_coupled_oscillators

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("viridis")

print("Libraries imported successfully!")

## 1. Create Ground Truth Network

We'll create a directed graph that represents the true causal relationships for our Poisson process.

In [ ]:
# Create a ground truth network
n_nodes = 6
seed = 42

# Create a network with interesting structure for Poisson processes
np.random.seed(seed)
G_true = nx.DiGraph()
G_true.add_nodes_from(range(n_nodes))

# Add causal edges that create interesting count dynamics
edges = [(0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 5)]
G_true.add_edges_from(edges)

print(f"Ground truth network has {G_true.number_of_nodes()} nodes and {G_true.number_of_edges()} edges")
print(f"Edges: {list(G_true.edges())}")

# Get adjacency matrix for later comparison
A_true = nx.adjacency_matrix(G_true).toarray()
print(f"\nGround truth adjacency matrix:")
print(A_true)

## 2. Generate Synthetic Poisson Time Series

Generate count data where each node's rate depends on its neighbors' previous counts.

In [ ]:
# Generate synthetic Poisson time series
T = 200  # Time series length
lambda_base = 3.0  # Base Poisson rate
coupling_strength = 0.5  # How much neighbors influence the rate

# Generate data using our custom Poisson generator
data, A_generated = poisson_coupled_oscillators(
    n=n_nodes,
    T=T,
    G=G_true,  # Use our predefined network
    lambda_base=lambda_base,
    coupling_strength=coupling_strength,
    seed=seed
)

print(f"Generated Poisson time series data with shape: {data.shape}")
print(f"Data statistics:")
print(f"  Mean: {np.mean(data):.3f}")
print(f"  Std:  {np.std(data):.3f}")
print(f"  Range: [{np.min(data):.0f}, {np.max(data):.0f}]")
print(f"  Data type: {data.dtype} (discrete counts)")

# Verify that generated adjacency matches our ground truth
print(f"\nAdjacency matrix match: {np.array_equal(A_true, A_generated.astype(int))}")

## 3. Visualize Poisson Time Series Data

Plot the count dynamics to understand the characteristics of Poisson data.

In [ ]:
# Plot time series for all variables
fig, axes = plt.subplots(n_nodes, 1, figsize=(14, 10), sharex=True)
fig.suptitle('Poisson Coupled Count Processes', fontsize=16, fontweight='bold')

time = np.arange(T)
colors = sns.color_palette("viridis", n_nodes)

for i in range(n_nodes):
    # Plot as step function to emphasize discrete nature
    axes[i].step(time, data[:, i], color=colors[i], alpha=0.8, linewidth=1.5, where='post')
    axes[i].fill_between(time, 0, data[:, i], color=colors[i], alpha=0.3, step='post')
    
    axes[i].set_ylabel(f'Count X{i}', fontweight='bold')
    axes[i].grid(True, alpha=0.3)
    
    # Add statistics
    mean_val = np.mean(data[:, i])
    axes[i].axhline(mean_val, color='red', linestyle='--', alpha=0.7, 
                   label=f'Mean: {mean_val:.1f}')
    axes[i].legend(fontsize=8, loc='upper right')
    
    # Set y-axis to show integer ticks
    axes[i].set_yticks(range(int(np.min(data[:, i])), int(np.max(data[:, i])) + 1, max(1, int(np.max(data[:, i])) // 5)))

axes[-1].set_xlabel('Time', fontweight='bold')
plt.tight_layout()
plt.show()

# Plot histogram of counts for each variable
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()
fig.suptitle('Distribution of Poisson Counts', fontsize=16, fontweight='bold')

for i in range(n_nodes):
    counts = data[:, i]
    axes[i].hist(counts, bins=range(int(np.min(counts)), int(np.max(counts)) + 2), 
                alpha=0.7, color=colors[i], edgecolor='black', linewidth=0.5)
    axes[i].set_title(f'X{i} (λ≈{np.mean(counts):.1f})', fontweight='bold')
    axes[i].set_xlabel('Count')
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)
    
    # Overlay theoretical Poisson distribution
    x_theory = np.arange(int(np.max(counts)) + 1)
    poisson_pmf = np.exp(-np.mean(counts)) * (np.mean(counts) ** x_theory) / np.array([np.math.factorial(x) for x in x_theory])
    axes[i].plot(x_theory, poisson_pmf * len(counts), 'r-', alpha=0.8, linewidth=2, 
                label=f'Poisson(λ={np.mean(counts):.1f})')
    axes[i].legend(fontsize=8)

plt.tight_layout()
plt.show()

## 4. Visualize Ground Truth Network

Display the true causal network structure that generates the count dynamics.

In [ ]:
# Plot ground truth network
plt.figure(figsize=(12, 8))

# Create layout that shows the flow structure
pos = nx.spring_layout(G_true, seed=seed, k=3, iterations=50)

# Draw network with emphasis on count-based coupling
node_sizes = [1500 + 200 * np.mean(data[:, i]) for i in range(n_nodes)]
nx.draw_networkx_nodes(G_true, pos, node_color='lightcoral', 
                       node_size=node_sizes, alpha=0.8)
nx.draw_networkx_edges(G_true, pos, edge_color='darkred', 
                       arrows=True, arrowsize=25, width=3, alpha=0.7)
nx.draw_networkx_labels(G_true, pos, {i: f'X{i}\n(λ≈{np.mean(data[:, i]):.1f})' for i in range(n_nodes)},
                        font_size=10, font_weight='bold')

plt.title('Ground Truth Causal Network\n(Poisson Count Data)\nNode size ∝ mean count rate', 
          fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

# Print network statistics
print("Ground Truth Network Statistics:")
print(f"  Nodes: {G_true.number_of_nodes()}")
print(f"  Edges: {G_true.number_of_edges()}")
print(f"  Edge density: {nx.density(G_true):.3f}")
print(f"  Is DAG: {nx.is_directed_acyclic_graph(G_true)}")
print(f"  Average count rates: {[f'X{i}: {np.mean(data[:, i]):.1f}' for i in range(n_nodes)]}")

## 5. Apply Causal Discovery with Poisson Method

Use the Poisson information method to discover causal relationships from the count data.

In [ ]:
# Apply causal discovery with Poisson method
print("Applying causal discovery with Poisson information method...")
print("This method is specifically designed for count data!\n")

# Test different discovery methods with Poisson information
methods_to_test = ['standard', 'alternative']
discovered_networks = {}

for method in methods_to_test:
    print(f"Running {method} method with Poisson information...")
    
    G_discovered = discover_network(
        data=data,
        method=method,
        information='poisson',  # Key: Use Poisson-specific information measure
        max_lag=2,
        alpha_forward=0.1,  # Slightly more lenient for count data
        alpha_backward=0.1,
        n_shuffles=100
    )
    
    discovered_networks[method] = G_discovered
    print(f"  Discovered {G_discovered.number_of_edges()} edges")
    print(f"  Edges: {list(G_discovered.edges())}\n")

## 6. Visualize Discovered Networks

Compare the discovered networks with the ground truth.

In [ ]:
# Plot comparison of networks
fig, axes = plt.subplots(1, len(methods_to_test) + 1, figsize=(6 * (len(methods_to_test) + 1), 6))
if len(methods_to_test) == 1:
    axes = [axes[0], axes[1]]

# Plot ground truth
ax = axes[0]
nx.draw_networkx_nodes(G_true, pos, node_color='lightcoral', 
                       node_size=1200, alpha=0.8, ax=ax)
nx.draw_networkx_edges(G_true, pos, edge_color='darkred', 
                       arrows=True, arrowsize=20, width=2.5, alpha=0.7, ax=ax)
nx.draw_networkx_labels(G_true, pos, {i: f'X{i}' for i in range(n_nodes)},
                        font_size=12, font_weight='bold', ax=ax)
ax.set_title('Ground Truth\n(Poisson Coupling)', fontweight='bold')
ax.axis('off')

# Plot discovered networks
colors = ['lightsteelblue', 'lightgreen']
edge_colors = ['darkblue', 'darkgreen']

for i, (method, G_disc) in enumerate(discovered_networks.items()):
    ax = axes[i + 1]
    
    # Convert node names back to integers
    G_disc_int = nx.DiGraph()
    G_disc_int.add_nodes_from(range(n_nodes))
    for edge in G_disc.edges():
        src = int(edge[0].replace('X', '')) if 'X' in str(edge[0]) else int(edge[0])
        dst = int(edge[1].replace('X', '')) if 'X' in str(edge[1]) else int(edge[1])
        G_disc_int.add_edge(src, dst)
    
    nx.draw_networkx_nodes(G_disc_int, pos, node_color=colors[i], 
                           node_size=1200, alpha=0.8, ax=ax)
    nx.draw_networkx_edges(G_disc_int, pos, edge_color=edge_colors[i], 
                           arrows=True, arrowsize=20, width=2.5, alpha=0.7, ax=ax)
    nx.draw_networkx_labels(G_disc_int, pos, {i: f'X{i}' for i in range(n_nodes)},
                            font_size=12, font_weight='bold', ax=ax)
    ax.set_title(f'Discovered\n({method})', fontweight='bold')
    ax.axis('off')

plt.suptitle('Network Comparison: Poisson Information Method', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Show edge comparison table
print("\nEDGE COMPARISON:")
print("="*40)
true_edges = set(G_true.edges())
print(f"Ground Truth Edges: {true_edges}")

for method, G_disc in discovered_networks.items():
    # Convert discovered edges to integer format
    disc_edges = set()
    for edge in G_disc.edges():
        src = int(edge[0].replace('X', '')) if 'X' in str(edge[0]) else int(edge[0])
        dst = int(edge[1].replace('X', '')) if 'X' in str(edge[1]) else int(edge[1])
        disc_edges.add((src, dst))
    
    print(f"{method.capitalize()} Discovered: {disc_edges}")
    
    # Calculate overlap
    correct = true_edges.intersection(disc_edges)
    missed = true_edges - disc_edges
    false_positive = disc_edges - true_edges
    
    print(f"  ✓ Correct: {correct}")
    print(f"  ✗ Missed: {missed}")
    print(f"  ⚠ False Pos: {false_positive}")
    print()

## 7. Calculate ROC-AUC Performance

Evaluate the performance of the Poisson method using ROC-AUC score.

In [ ]:
def calculate_roc_auc_poisson(true_adj, discovered_graph):
    """Calculate ROC-AUC for Poisson method network discovery."""
    n = true_adj.shape[0]
    
    # Convert discovered graph to adjacency matrix
    G_int = nx.DiGraph()
    G_int.add_nodes_from(range(n))
    for edge in discovered_graph.edges():
        src = int(edge[0].replace('X', '')) if 'X' in str(edge[0]) else int(edge[0])
        dst = int(edge[1].replace('X', '')) if 'X' in str(edge[1]) else int(edge[1])
        G_int.add_edge(src, dst)
    
    discovered_adj = nx.adjacency_matrix(G_int, nodelist=range(n)).toarray()
    
    # Flatten and remove diagonal
    mask = ~np.eye(n, dtype=bool).flatten()
    y_true = true_adj.flatten()[mask]
    y_scores = discovered_adj.flatten()[mask]
    
    # Calculate ROC-AUC
    if len(np.unique(y_true)) > 1:
        auc_score = roc_auc_score(y_true, y_scores)
        fpr, tpr, _ = roc_curve(y_true, y_scores)
        return auc_score, fpr, tpr
    else:
        return None, None, None

# Calculate ROC-AUC for each method
results = {}
plt.figure(figsize=(10, 6))

colors_roc = ['blue', 'green']
for i, (method, G_disc) in enumerate(discovered_networks.items()):
    auc_score, fpr, tpr = calculate_roc_auc_poisson(A_true, G_disc)
    
    if auc_score is not None:
        results[method] = {
            'auc': auc_score,
            'fpr': fpr,
            'tpr': tpr
        }
        
        # Plot ROC curve
        plt.plot(fpr, tpr, color=colors_roc[i], linewidth=3, 
                label=f'{method} (AUC = {auc_score:.3f})')
        
        print(f"{method.upper()} METHOD (Poisson Information):")
        print(f"  ROC-AUC Score: {auc_score:.3f}")
        print(f"  Interpretation: {'Excellent' if auc_score > 0.9 else 'Good' if auc_score > 0.7 else 'Fair' if auc_score > 0.6 else 'Poor'}")
        print()
    else:
        print(f"{method} method: Cannot calculate AUC (insufficient variation)")

# Plot random performance line
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, linewidth=2, label='Random (AUC = 0.500)')

plt.xlabel('False Positive Rate', fontweight='bold', fontsize=12)
plt.ylabel('True Positive Rate', fontweight='bold', fontsize=12)
plt.title('ROC Curves for Poisson Causal Discovery\n(Count Data with Poisson Information Method)', 
          fontweight='bold', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "="*55)
print("PERFORMANCE SUMMARY - POISSON METHOD")
print("="*55)
print(f"Ground truth edges: {np.sum(A_true)}")
print(f"Data characteristics: Count data (mean rate ≈ {np.mean(data):.1f})")
print(f"Information method: Poisson (optimal for count data)")
print()
for method, G_disc in discovered_networks.items():
    print(f"{method.capitalize()} method results:")
    print(f"  Discovered edges: {G_disc.number_of_edges()}")
    if method in results:
        print(f"  ROC-AUC: {results[method]['auc']:.3f}")
        print(f"  Performance: {'🟢 Excellent' if results[method]['auc'] > 0.9 else '🟡 Good' if results[method]['auc'] > 0.7 else '🟠 Fair' if results[method]['auc'] > 0.6 else '🔴 Poor'}")
    print()

## 8. Detailed Performance Analysis

Calculate comprehensive performance metrics for the Poisson method.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

def detailed_performance_analysis_poisson(true_adj, discovered_graph):
    """Calculate detailed performance metrics for Poisson method."""
    n = true_adj.shape[0]
    
    # Convert discovered graph to adjacency matrix
    G_int = nx.DiGraph()
    G_int.add_nodes_from(range(n))
    for edge in discovered_graph.edges():
        src = int(edge[0].replace('X', '')) if 'X' in str(edge[0]) else int(edge[0])
        dst = int(edge[1].replace('X', '')) if 'X' in str(edge[1]) else int(edge[1])
        G_int.add_edge(src, dst)
    
    discovered_adj = nx.adjacency_matrix(G_int, nodelist=range(n)).toarray()
    
    # Flatten and remove diagonal
    mask = ~np.eye(n, dtype=bool)
    y_true = true_adj[mask]
    y_pred = discovered_adj[mask]
    
    # Calculate metrics
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'false_negatives': fn,
        'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0,
        'accuracy': (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
    }

# Calculate detailed metrics
print("\nDETAILED PERFORMANCE ANALYSIS - POISSON METHOD")
print("="*60)
print("📊 This analysis shows how well Poisson information")
print("   handles count data compared to generic methods.\n")

for method, G_disc in discovered_networks.items():
    metrics = detailed_performance_analysis_poisson(A_true, G_disc)
    
    print(f"🔍 {method.upper()} METHOD WITH POISSON INFORMATION:")
    print(f"   Precision:    {metrics['precision']:.3f} (What fraction of discovered edges are correct?)")
    print(f"   Recall:       {metrics['recall']:.3f} (What fraction of true edges were found?)")
    print(f"   F1-Score:     {metrics['f1_score']:.3f} (Harmonic mean of precision & recall)")
    print(f"   Specificity:  {metrics['specificity']:.3f} (True negative rate)")
    print(f"   Accuracy:     {metrics['accuracy']:.3f} (Overall correctness)")
    print(f"   ")  
    print(f"   Confusion Matrix Details:")
    print(f"   ✓ True Positives:  {metrics['true_positives']} (correctly found edges)")
    print(f"   ✗ False Positives: {metrics['false_positives']} (incorrectly added edges)")
    print(f"   ✓ True Negatives:  {metrics['true_negatives']} (correctly absent edges)")
    print(f"   ✗ False Negatives: {metrics['false_negatives']} (missed true edges)")
    
    if method in results:
        print(f"   📈 ROC-AUC:       {results[method]['auc']:.3f}")
    print("\n" + "-"*50 + "\n")

# Compare with what we'd expect from random discovery
n_possible_edges = n_nodes * (n_nodes - 1)  # Exclude self-loops
random_precision = G_true.number_of_edges() / n_possible_edges
print(f"📋 BASELINE COMPARISON:")
print(f"   Random precision would be: {random_precision:.3f}")
print(f"   Total possible directed edges: {n_possible_edges}")
print(f"   Ground truth edges: {G_true.number_of_edges()}")

## 9. Why Poisson Method Works for Count Data

Let's examine why the Poisson information method is particularly suitable for count data.

In [ ]:
# Analyze data characteristics that make Poisson method appropriate
print("🧮 POISSON METHOD SUITABILITY ANALYSIS")
print("="*50)

# Check Poisson assumptions
print("📈 DATA CHARACTERISTICS:")
for i in range(n_nodes):
    counts = data[:, i]
    mean_count = np.mean(counts)
    var_count = np.var(counts)
    ratio = var_count / mean_count if mean_count > 0 else 0
    
    print(f"   X{i}: Mean={mean_count:.2f}, Var={var_count:.2f}, Var/Mean={ratio:.2f}")
    print(f"       {'✓ Good Poisson fit' if 0.8 <= ratio <= 1.2 else '⚠ Overdispersed' if ratio > 1.2 else '⚠ Underdispersed'}")

print(f"\n🎯 WHY POISSON METHOD IS EFFECTIVE HERE:")
print(f"   • Data are discrete counts (integers ≥ 0)")
print(f"   • Variance ≈ mean for most variables (Poisson property)")
print(f"   • Coupling affects rate parameters (biologically realistic)")
print(f"   • Poisson mutual information captures count dependencies")

print(f"\n📊 DATA SUMMARY:")
print(f"   Time series length: {T}")
print(f"   Number of count processes: {n_nodes}")
print(f"   Base rate (λ): {lambda_base}")
print(f"   Coupling strength: {coupling_strength}")
print(f"   Data range: [{np.min(data):.0f}, {np.max(data):.0f}]")
print(f"   All values are non-negative integers: {np.all(data >= 0) and np.all(data == np.round(data))}")

# Visualization of Poisson fit
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Variance vs Mean (should be close to diagonal for Poisson)
means = [np.mean(data[:, i]) for i in range(n_nodes)]
variances = [np.var(data[:, i]) for i in range(n_nodes)]

axes[0].scatter(means, variances, s=100, alpha=0.7, color='red')
axes[0].plot([0, max(means)], [0, max(means)], 'k--', alpha=0.5, label='Var = Mean (Perfect Poisson)')
for i in range(n_nodes):
    axes[0].annotate(f'X{i}', (means[i], variances[i]), xytext=(5, 5), textcoords='offset points')
axes[0].set_xlabel('Mean Count', fontweight='bold')
axes[0].set_ylabel('Variance', fontweight='bold')
axes[0].set_title('Poisson Assumption Check\n(Variance vs Mean)', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Distribution comparison for one variable
i_example = 0  # Show first variable as example
counts = data[:, i_example]
unique_counts = np.arange(int(np.max(counts)) + 1)

# Observed frequencies
observed_freq = np.bincount(counts.astype(int), minlength=len(unique_counts))
observed_prob = observed_freq / len(counts)

# Theoretical Poisson probabilities
lambda_est = np.mean(counts)
theoretical_prob = np.exp(-lambda_est) * (lambda_est ** unique_counts) / np.array([np.math.factorial(x) for x in unique_counts])

x_pos = np.arange(len(unique_counts))
width = 0.35
axes[1].bar(x_pos - width/2, observed_prob, width, label=f'Observed X{i_example}', alpha=0.7, color='blue')
axes[1].bar(x_pos + width/2, theoretical_prob, width, label=f'Poisson(λ={lambda_est:.1f})', alpha=0.7, color='orange')

axes[1].set_xlabel('Count Value', fontweight='bold')
axes[1].set_ylabel('Probability', fontweight='bold')
axes[1].set_title(f'Distribution Comparison: X{i_example}\n(Observed vs Theoretical Poisson)', fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(unique_counts)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Conclusions

Summary of the Poisson causal discovery experiment and key insights.

In [ ]:
print("\n" + "="*65)
print("🎯 EXPERIMENT CONCLUSIONS - POISSON CAUSAL DISCOVERY")
print("="*65)

print(f"\n📊 DATA CHARACTERISTICS:")
print(f"  • Time series length: {T}")
print(f"  • Number of count processes: {n_nodes}")
print(f"  • Ground truth edges: {G_true.number_of_edges()}")
print(f"  • Data type: Discrete counts (Poisson-distributed)")
print(f"  • Base rate: λ = {lambda_base}")
print(f"  • Coupling mechanism: Rate modulation by neighbor counts")

print(f"\n🔍 DISCOVERY RESULTS:")
best_method = None
best_auc = 0

for method, G_disc in discovered_networks.items():
    auc_val = results.get(method, {}).get('auc', 0)
    performance_level = ('🟢 Excellent' if auc_val > 0.9 else 
                        '🟡 Good' if auc_val > 0.7 else 
                        '🟠 Fair' if auc_val > 0.6 else '🔴 Poor')
    print(f"  • {method.capitalize()} method: {G_disc.number_of_edges()} edges discovered")
    print(f"    ROC-AUC = {auc_val:.3f} {performance_level}")
    
    if auc_val > best_auc:
        best_auc = auc_val
        best_method = method

print(f"\n🏆 BEST PERFORMING METHOD: {best_method.upper() if best_method else 'None'}")
if best_method and best_method in results:
    print(f"  • ROC-AUC Score: {best_auc:.3f}")
    print(f"  • Edges discovered: {discovered_networks[best_method].number_of_edges()}")
    print(f"  • Performance level: {'🟢 Excellent' if best_auc > 0.9 else '🟡 Good' if best_auc > 0.7 else '🟠 Fair' if best_auc > 0.6 else '🔴 Poor'}")

print(f"\n💡 KEY INSIGHTS:")
print(f"  • ✅ Poisson method is specifically designed for count data")
print(f"  • ✅ Captures dependencies in discrete event processes")
print(f"  • ✅ Handles rate-based coupling mechanisms naturally")
print(f"  • ⚠️  Performance depends on signal-to-noise ratio in coupling")
print(f"  • ⚠️  May struggle with very sparse counts or overdispersion")

print(f"\n🧮 POISSON METHOD ADVANTAGES:")
print(f"  • Respects discrete nature of count data")
print(f"  • Uses appropriate probability distributions")
print(f"  • Better than Gaussian methods for count processes")
print(f"  • Captures rate dependencies effectively")

print(f"\n📈 WHEN TO USE POISSON METHOD:")
print(f"  • Data are non-negative integer counts")
print(f"  • Variance approximately equals mean")
print(f"  • Process involves event counting or rates")
print(f"  • Examples: Gene expression, neural spikes, web clicks")

print(f"\n📝 RECOMMENDATIONS:")
print(f"  • Use Poisson method for any count-based time series")
print(f"  • Check variance/mean ratio to validate Poisson assumption")
print(f"  • Consider negative binomial method if data is overdispersed")
print(f"  • Ensure sufficient data length for reliable discovery")

print("\n🎉 Poisson causal discovery experiment completed successfully!")
print("   This method provides a principled approach for count data analysis.")